In [208]:
import pandas as pd
import numpy as np
import os
import requests
import json
import googleapiclient.discovery
import altair as alt

In [209]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## Data acquisition

#### Getting a list with videos

List of videos using the YouTube Data API [YouTube Data API](https://tools.digitalmethods.net/netvizz/youtube/mod_videos_list.php)

Querying for the terms: `Global warming`, `Climate change`, `Paris agreement`, `Climate realism`.

#### Getting all comments (including replies) to all videos in the list

Get all comments to a video using the [CommentThreads method of YouTube Developer API](https://developers.google.com/youtube/v3/docs/commentThreads/list)

The API documentation of CommentsThread states that it might not contain all replies: 

>A commentThread resource contains information about a YouTube comment thread, which comprises a top-level comment and replies, if any exist, to that comment. A commentThread resource can represent comments about either a video or a channel.

>Both the top-level comment and the replies are actually comment resources nested inside the commentThread resource. The commentThread resource does not necessarily contain all replies to a comment, and you need to use the comments.list method if you want to retrieve all replies for a particular comment. Also note that some comments do not have replies.

Therefore we use the [Coments list method](https://developers.google.com/youtube/v3/docs/commentThreads/list) to get all replies to a comment. 

In [210]:
API_KEY = enter here your api key

#### List of vidoes containing the term `Paris agreement`

In [211]:
data_path = 'data_raw/videolist_search50_2021_01_19-13_55_33.tab'

In [212]:
videos = pd.read_csv(data_path,sep='\t',header=(0))
videos.head()

,position,channelId,channelTitle,videoId,publishedAt,publishedAtSQL,videoTitle,videoDescription,tags,videoCategoryId,videoCategoryLabel,duration,durationSec,dimension,definition,caption,thumbnail_maxres,licensedContent,viewCount,likeCount,dislikeCount,favoriteCount,commentCount
0,1,UC3XTzVzaHQEd30rQbuvCtTQ,LastWeekTonight,5scez5dqtAc,2017-06-05T06:30:00Z,2017-06-05 06:30:00,Paris Agreement: Last Week Tonight with John Oliver (HBO),Donald Trump plans to withdraw the United States from the Paris agreement on climate change. That's bad news for anyone who happens to live on this planet. Connect with Last Week Tonight online... Subscribe to the Last Week Tonight YouTube channel for more almost news as it almost happens: www.youtube.com/user/LastWeekTonight Find Last Week Tonight on Facebook like your mom would: http://Facebook.com/LastWeekTonight Follow us on Twitter for news about jokes and jokes about news: http://Twitter.com/LastWeekTonight Visit our official site for all that other stuff at once: http://www.hbo.com/lastweektonight,"last week tonight paris agreement,paris accord,john oliver paris agreement",24,Entertainment,PT20M58S,1258,2d,hd,False,https://i.ytimg.com/vi/5scez5dqtAc/maxresdefault.jpg,1.0,13021725,176853,12626,0,13747.0
1,2,UCuLEr-GWiwCBB6zBDX3elOQ,UN Climate Change,WiGD0OgK2ug,2020-09-24T16:19:23Z,2020-09-24 16:19:23,"Ever wondered: What is the 'Paris Agreement', and how does it work?","This video explains the basics of the Paris Agreement: the international UN treaty of 2015 that aims to tackle climate change head on. It's central aim is to strengthen the global response to the threat of climate change by keeping a global temperature rise this century well below 2 degrees Celsius above pre-industrial levels, with ambition to limit the increase even further to 1.5 degrees Celsius. Additionally, the agreement aims to strengthen the ability of countries to deal with the impacts of climate change.","Paris agreement,unfccc,United Nations climate change,climate change,Climate ambition,UN treaty,United Nations",25,News & Politics,PT1M40S,100,2d,hd,False,https://i.ytimg.com/vi/WiGD0OgK2ug/maxresdefault.jpg,NaN,21031,93,5,0,6.0
2,3,UCAuUUnT6oDeKwE6v1NGQxug,TED,MIA_1xQc7x8,2016-05-11T15:15:47Z,2016-05-11 15:15:47,The inside story of the Paris climate agreement | Christiana Figueres,"What would you do if your job was to save the planet? When Christiana Figueres was tapped by the UN to lead the Paris climate conference (COP 21) in December 2015, she reacted the way many people would: she thought it would be impossible to bring the leaders of 195 countries into agreement on how to slow climate change. Find out how she turned her skepticism into optimism — and helped the world achieve the most important climate agreement in history. TEDTalks is a daily video podcast of the best talks and performances from the TED Conference, where the world's leading thinkers and doers give the talk of their lives in 18 minutes (or less). Look for talks on Technology, Entertainment and Design -- plus science, business, global issues, the arts and much more. Find closed captions and translated subtitles in many languages at http://www.ted.com/translate Follow TED news on Twitter: http://www.twitter.com/tednews Like TED on Facebook: https://www.facebook.com/TED Subscribe to our channel: http://www.youtube.com/user/TEDtalksDirector","TED Talk,TED Talks,Christiana Figueres,UNFCCC,COP 21,Paris Agreement,climate change",29,Nonprofits & Activism,PT14M51S,891,2d,hd,True,https://i.ytimg.com/vi/MIA_1xQc7x8/maxresdefault.jpg,1.0,205893,2786,635,0,537.0
3,4,UC8exuytmPBqeM6klg7zBNsQ,Mindscape,1DdfNU5iATU,2017-06-10T21:42:52Z,2017-06-10 21:42:52,Paris agreement simplified,"Everything you need to know about Paris agreement is given here. which is easy to understand and gives at most clarity about this agreement. involvement of different countries . we have explained how actually the Paris agreement is going to work Supported by : Designed by Freepik"" De

In [213]:
comments_vid2 = pd.read_csv('data_raw/videoinfo_MIA_1xQc7x8_2021_01_20-12_13_25_comments.tab',sep='\t',header=(0))

We need to find more videos with more than a minimum number of comments. E.g. 2 comments.

In [215]:
index_names = videos[(videos['commentCount'] < 2)].index 
videos.drop(index_names, inplace = True) 

6


In [231]:
np.sum(videos['commentCount'])
print(f"We have in total {np.sum(videos['commentCount'])} from comments distributed in {len(videos)} for videos containing the term Paris Agreement.")

We have in total 33282.0 from comments distributed in 44 for videos containing the term Paris Agreement.


In [164]:
video_id = 'HzLEYJE33ww'

#### Class to load all comments of a video 

In [165]:
class Video_comments:
    
    def __init__(self, api_key, video_id):
        self.api_key  = api_key
        self.video_id = video_id
        self.max_results = 100
        
        comments_df = pd.DataFrame({
                            'id':[],
                            'published_at': [], 
                            'author_name': [], 
                            'text': [],
                            'is_reply': [],
                            'likeCount': [],
                            'cleaned': [],
                            'video_id': [],
                            'video_published_at': []}, 
                            columns = [ 'id',
                                        'published_at', 
                                        'author_name', 
                                        'text', 
                                        'likeCount',
                                        'is_reply', 
                                        'cleaned', 
                                        'video_id', 
                                        'video_published_at'])
        
        self.comments_df = comments_df
        
    
    '''load all replies of top level comments and append dataframe witth all top level comments and replies. 
    (appendingt to df and loading replies should be devided to different methods.)'''
    def _add_to_dataframe(self, response):
        for i, main_comment in enumerate(response['items']):
            comment = main_comment['snippet']['topLevelComment']['snippet']

            new_row = pd.Series(data={
                                    'id':main_comment['snippet']['topLevelComment']['id'],
                                    'published_at':comment['publishedAt'] , 
                                    'author_name': comment['authorDisplayName'], 
                                    'text': comment['textOriginal'],
                                    'likeCount':comment['likeCount'],
                                    'is_reply': 0,
                                    'video_id': comment['videoId']})

            self.comments_df = self.comments_df.append(new_row, ignore_index=True)

            
            #check if the top level comment has replies. If yey then get these too and add to df
            request_replies = requests.get(f"https://youtube.googleapis.com/youtube/v3/comments?part=snippet&parentId={main_comment['snippet']['topLevelComment']['id']}&key={self.api_key}")
            response_replies = json.loads(request_replies.text)
        
            #if response_replies['items'] > 0 then the main comment has replies
            if(len(response_replies['items']) > 0):
                for i, main_comment in enumerate(response_replies['items']):      
                    reply = main_comment['snippet']

                    new_row = pd.Series(data={
                                            'id':reply['parentId'],
                                            'published_at':reply['publishedAt'] , 
                                            'author_name': reply['authorDisplayName'], 
                                            'text': reply['textOriginal'],
                                            'likeCount':reply['likeCount'],
                                            'is_reply': 1,
                                            'video_id': comment['videoId']})

                    self.comments_df = self.comments_df.append(new_row, ignore_index=True)
                    
    
    '''Load (and append comments dataframe) recursively comments from next page until there are no next page. '''
    def _get_next_page(self, response):     
        request1 = requests.get(f"https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults={self.max_results}&pageToken={str(response['nextPageToken'])}&videoId={self.video_id}&key={self.api_key}")
        response1 = json.loads(request1.text)
        self._add_to_dataframe(response1)
        
        print('there is still a next page ',len(response1['items']))
        
        if ('nextPageToken' in response1.keys()):
            self._get_next_page(response1)
    
    '''Start loading comments. Paginated.'''
    def get_comments(self):    
        request = requests.get(f"https://www.googleapis.com/youtube/v3/commentThreads?part=snippet&maxResults={self.max_results}&videoId={self.video_id}&key={self.api_key}")
        response = json.loads(request.text)     
        
        print('first page ',len(response['items']))
        self._add_to_dataframe(response)
        
        if 'nextPageToken' in response.keys():
            print('there is a next page')
            self._get_next_page(response)
        
        return self.comments_df

In [166]:
vid_comments = Video_comments(API_KEY, video_id)

In [167]:
comments_df = vid_comments.get_comments()

first page  98


In [168]:
comments_df.shape

(221, 9)

In [169]:
comments_df

,id,published_at,author_name,text,likeCount,is_reply,cleaned,video_id,video_published_at
0,UgwCGr2EzEL9dMXnjSp4AaABAg,2021-01-21T01:48:03Z,Chewie 13,"Get ready for small windows, and a bankrupt country 2021-2029",0.0,0.0,NaN,HzLEYJE33ww,NaN
1,Ugxa6wli7G9yyFs7ImB4AaABAg,2021-01-21T00:34:17Z,Bradley Butters,Paris climate agreement is bad for America they are using ur taxes money that why taxes is 43% night taxes,0.0,0.0,NaN,HzLEYJE33ww,NaN
2,Ugxk4PWhOxoPtmf6EIV4AaABAg,2021-01-19T20:24:10Z,Ocean Energy,"WE CAN BE ALL HAPPY THAT YOU LEAVING MR. TRUMP ! TAKE CARE AND NEVER COME BACK !\n\nThe United States is the top oil-producing country in the world !\n\nThe top five largest oil producers are the following countries:\r\nUnited States. The United States is the top oil-producing country in the world, with an average of 19.47 million barrels per day (b/d), which accounts for 19% of the world's production. ...\r\n\nSaudi Arabia. ...\r\nRussia. ...\r\nCanada. ...\r\nChina.\r\nJan 1, 2021\n\nIT IS TIME THAT THE WORLDWIDE OILCOMPANIES PAY FOR THE WORLDWIDE DEMAGE !\n\nDESTROYING THE WORLD WITH FOSSIL FUELS IS A STUPID THING - WE ALL HAVE LEARNED !\n\nNOW IT IS TIME TO ACT IN THE RIGHT WAY TO CONTROL THE CLIMATE CHANGE !\n\nEVERY USD, EURO OR CURRENCY FROM WORLDWIE COUNTRIES IS NEEDED TO FIX EARTH BEFORE IT IS TO LATE !",0.0,0.0,NaN,HzLEYJE33ww,NaN
3,Ugx9PNTogkD2l2lKGeR4AaABAg,2021-01-19T01:27:02Z,Nicholas Colding,I'm so glad Mr.Trump lost that election.,0.0,0.0,NaN,HzLEYJE33ww,NaN
4,Ugy1NwNspTaF5mtkr2F4AaABAg,2020-12-25T06:41:25Z,P.R Bones,Cap,2.0,0.0,NaN,HzLEYJE33ww,NaN
5,Ugzn1QUv5ERAVysw0IN4AaABAg,2020-12-22T03:29:07Z,Rain Tang,How to down load this,0.0,0.0,NaN,HzLEYJE33ww,NaN
6,Ugw90YcQlAzKZd-WIB94AaABAg,2020-12-19T17:41:11Z,Lisa Longo,Trump was 100% right!,1.0,0.0,NaN,HzLEYJE33ww,NaN
7,Ugw4mmMNhUNvqJj8JFZ4AaABAg,2020-12-02T23:34:42Z,t w,lol just because trump pulled out of paris accord doesn't mean he is against climate. stupid idiots.,0.0,0.0,NaN,HzLEYJE33ww,NaN
8,UgysnlK5Pb-Jd8y-xkh4AaABAg,2020-11-24T22:27:37Z,reference2me,Check out the Paris agreement yourself ... the USA gets the bill and the money goes to the Africa elite ...,3.0,0.0,NaN,HzLEYJE33ww,NaN
9,UgxiaOrJeIokS5zmxoV4AaABAg,2020-11-23T16:52:19Z,Cowman 979,This is going to be undone now Biden won,7.0,0.0,NaN,HzLEYJE33ww,NaN
